In [71]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import connections
client = Elasticsearch()

connections.configure(
    default={'hosts': 'localhost'},
)

In [72]:
from elasticsearch_dsl import analyzer
analyzer = analyzer(
    'my_tokenizer',
    tokenizer="standard",
    filter=["morfologik_stem"],
)

In [73]:
from elasticsearch_dsl import Index
judgments = Index('judgments')
judgments.analyzer(analyzer)
judgments.create()
# judgments.delete()


In [74]:
from elasticsearch_dsl import DocType, Text, Date, Keyword, Nested, InnerDoc

class Judge(InnerDoc):
    name = Text(analyzer=analyzer)

@judgments.doc_type
class Judgment(DocType):
    content = Text(analyzer=analyzer)
    judgment_date = Date()
    signature = Keyword()
    judges = Nested(Judge)

    class Meta:
        index = 'judgments'


In [75]:
# configs 
DATA_DIR = "/run/media/maciej/Nowy/data/json/"
CHOSEN_YEAR = str(2011)
FILE_LIST = 'good_files.txt'


In [76]:
import json
import os
import pickle

from tqdm import tqdm

def load_data():    
    files = pickle.load(open(FILE_LIST, 'rb'))
    results = []
    for file in tqdm(files):
        if file.startswith("judgment"):
            file_path = os.path.join(DATA_DIR, file)

            with open(file_path, 'r') as f:
                data = json.load(f)
                judgments = [x for x in data["items"] if x["judgmentDate"].startswith(CHOSEN_YEAR)]

            for judgment in judgments:
                Judgment(
                    content=judgment['textContent'],
                    judgment_date=judgment['judgmentDate'],
                    signature=judgment['id'],
                    judge=[Judge(name=judge['name']) for judge in judgment['judges']],
                ).save()
    return results


load_data()


  0%|          | 0/68 [00:00<?, ?it/s]
Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/lib64/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/maciej/.virtualenvs/nlp/lib/python3.6/site-packages/tqdm/_tqdm.py", line 148, in run
    for instance in self.tqdm_cls._instances:
  File "/home/maciej/.virtualenvs/nlp/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|██████████| 68/68 [01:05<00:00,  1.04it/s]


[]

In [84]:
s = Judgment.search()
s.query('match').execute()

GET http://localhost:9200/judgments/doc/_search [status:400 request:0.004s]


RequestError: TransportError(400, 'parsing_exception', 'No text specified for text query')